In [1]:
import cv2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
from keras.preprocessing import image as Kimage
from keras.utils import np_utils
from keras.layers import BatchNormalization
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint  
from keras import optimizers

from tqdm import tqdm
import pydicom
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
import os
print(os.listdir("../input/"))


/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


['stage_1_detailed_class_info.csv', 'stage_1_sample_submission.csv', 'stage_1_test_images', 'stage_1_train_labels.csv', 'stage_1_train_images', 'GCP Credits Request Link - RSNA.txt']


Below is the list of parameters that can be configured before running the model
* model_type: String, FC512|FC1024|FC
* dropout: Probability
* optimizer_type: String, Adam|SGD|Adadelta default RMS
* learning_rate: learing rate for optimizer
* Augmentation_Indicator: Boolean, Implement Augmentaiton or not
* epochs = Int
* batch_size = Int
* transfer_learning = Boolean, Repsective transfer learnign has to be imported and edited in code
* random_state = random value
* resize=Boolean
* input_shape=tuple, Resize shape (256,256,3)
* rescale=Int
* sample_ratio = any value 0 to 1, Limiting the input data

In [2]:
model_type = 'FC1024'
dropout = 0.4
optimizer_type = 'Adam'
learning_rate = 1e-4
Augmentation_Indicator = False
epochs = 15
batch_size = 16
transfer_learning = True
random_state = 1607
resize=True
input_shape=(256, 256, 3)
rescale=100
sample_ratio = 0.35

In [3]:
input_shape[0:2]

(256, 256)

In [4]:
# Load train labels
input_data = pd.read_csv("../input/stage_1_detailed_class_info.csv")
input_data['img_path'] = '../input/stage_1_train_images/' + input_data['patientId'] + '.dcm'

# Convert class into categorical variable
input_data['class'] = pd.Categorical(input_data['class'])
input_data['target'] = input_data['class'].cat.codes

# Start with around 1500 - 2000 images 
# This step is not needed when it is going to be trained with full resources 
remove, input_data  = train_test_split(input_data, 
                                test_size=sample_ratio, 
                                random_state=random_state,
                                stratify=input_data['class'])

print('Total images taken: {}'.format(input_data.shape[0]))
total_images=input_data.shape[0]
input_data.head()

Total images taken: 10147


,patientId,class,img_path,target
12429,7ed2ee08-c7e0-40c5-afe8-d6327df0a5ee,No Lung Opacity / Not Normal,../input/stage_1_train_images/7ed2ee08-c7e0-40...,1
4734,40f55d75-0e89-4392-a32c-84ff66530a4a,No Lung Opacity / Not Normal,../input/stage_1_train_images/40f55d75-0e89-43...,1
26939,efc5ae9c-b54b-4cf3-a448-403241ec3e03,No Lung Opacity / Not Normal,../input/stage_1_train_images/efc5ae9c-b54b-4c...,1
17405,a67be9a7-9130-4e1b-a7a1-1f137ffddb7d,Lung Opacity,../input/stage_1_train_images/a67be9a7-9130-4e...,0
4991,432be6a7-fb99-4eec-827c-deb0310b8967,No Lung Opacity / Not Normal,../input/stage_1_train_images/432be6a7-fb99-4e...,1


Split the data-set into train, valida and test.
Train: 72%, Valid: 18%, Test: 10%

In [5]:
# Split train and test images
train, test = train_test_split(input_data, 
                                test_size=0.1, 
                                random_state=random_state,
                                stratify=input_data['class'])

# Split train and validation images
train, valid = train_test_split(train, 
                                test_size=0.2, 
                                random_state=random_state,
                                stratify=train['class'])

print('Total train images taken: {}'.format(train.shape[0]))
print('Total validation images taken: {}'.format(valid.shape[0]))
print('Total test images taken: {}'.format(test.shape[0]))


Total train images taken: 7305
Total validation images taken: 1827
Total test images taken: 1015


The below functions help to read an image and convert into tensor required for keras processing

In [6]:
def load_dicom_image(img_path):
    """
    This function takes the image path and reads the DICOM image
    :param img_path: image path
    :return: pil Image
    """
    img_arr = pydicom.read_file(img_path).pixel_array
    img_arr = img_arr/img_arr.max()
    slice_value = (255*img_arr).clip(0, 255).astype(np.uint8)
    img = Image.fromarray(slice_value)
    Kimage.pil_image = img
    return Kimage.pil_image

def load_resized_image(img_path):
    """
    This function converts the image into new shape as set in parameters
    :param imag_path: image path
    :return: image array
    """
    ds = pydicom.dcmread(img_path)
    resized_image = cv2.resize(ds.pixel_array, input_shape[0:2])
    resized_image = np.repeat(resized_image[:, :, np.newaxis], 3, axis=2)
    return resized_image.astype(np.float32)

# Convert 3D tensors to 4D tensors where each 4D tensor is a different image
def path_to_tensor(img_path, resize=False):
    """
    This function reads 2D array
    :param imag_path: image path
    :return: 3D Tensor
    """
    if resize==True:
        x = load_resized_image(img_path)      
    else:
        # Read the dcm image using pydicom
        img = load_dicom_image(img_path)
        # convert PIL.Image.Image type to 3D tensor
        x = Kimage.img_to_array(img)
        # Since it is a grayscale image convert into three channels
        x = np.squeeze(np.repeat(x[:, :, np.newaxis], 3, axis=2), axis=3)
    # convert 3D tensor to 4D tensor with shape and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    """
    This function reads 3D arrays and returns the 4D Tensor
    :param imag_path: image paths of all 3D arrays
    :return: 4D Tensor
    """
    list_of_tensors = [preprocess_input(path_to_tensor(img_path, resize)) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)


In [7]:
# Load all the tensors and re-scale the data if requried
train_tensors = paths_to_tensor(train['img_path'])*rescale
valid_tensors = paths_to_tensor(valid['img_path'])*rescale
test_tensors = paths_to_tensor(test['img_path'])*rescale


100%|██████████| 1015/1015 [00:12<00:00, 83.45it/s]


In [8]:
# Load all the targets
train_targets = np_utils.to_categorical(np.array(train['target']), 3)
valid_targets = np_utils.to_categorical(np.array(valid['target']), 3)
test_targets = np_utils.to_categorical(np.array(test['target']), 3)

In [9]:
train_tensors.shape

(7305, 256, 256, 3)

The below step will downlaod the trasnfer learning model from Keras and load the weights. This will be the base model if transfer learning is chose. This model can not be trained due to limitation of resources

In [10]:
if transfer_learning:
    # Load VGG/Xception model from keras
    base_model = VGG19(input_shape=input_shape, weights='imagenet', include_top=False)
    base_model.trainable = False
    output_shape_trf_learning=base_model.get_output_shape_at(0)[1:]
    learning_name = 'VGG19TransferLearning'
    base_model.summary()
else:
    input_shape=input_shape
    learning_name = 'OwnCNN'

80142336/80134624 [==============================] - 2s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool

In [11]:
output_shape_trf_learning

(8, 8, 512)

Below are differnet experimentations of the top layer and this will be seconf part of the trasnfer learning.

In [12]:
if model_type == 'FC1024':
    # Build the final layer of the model
    model = Sequential()

    model.add(Conv2D(filters=1024, kernel_size=2, padding='same', activation='relu', input_shape=output_shape_trf_learning))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='tanh'))
    model.add(Dropout(dropout))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=256, kernel_size=2, padding='same', activation='relu'))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=256, kernel_size=2, padding='same', activation='relu'))
    model.add(Dropout(dropout))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(3, activation='softmax'))

    model.summary()
elif model_type == 'FC512':
    model = Sequential()

    model.add(Conv2D(filters=1024, kernel_size=2, padding='same', activation='relu', input_shape=output_shape_trf_learning))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='tanh'))
    model.add(Dropout(dropout))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=256, kernel_size=2, padding='same', activation='relu'))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(dropout))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(3, activation='softmax'))

    model.summary()
elif model_type == 'FC16':
    model = Sequential()

    model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=output_shape_trf_learning))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(dropout))
    model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='tanh'))
    model.add(Dropout(dropout))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
    model.add(Dropout(dropout))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
    model.add(Dropout(dropout))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(3, activation='softmax'))

    model.summary()
elif model_type == 'FCTest':
    model = Sequential()

    model.add(Dense(1024, activation='relu', input_shape=output_shape_trf_learning))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(512,activation='tanh'))
    model.add(Dropout(dropout))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(dropout))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(3, activation='softmax'))

    model.summary()
else:
    model = Sequential()
    model.add(BatchNormalization(input_shape=output_shape_trf_learning))
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(dropout))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(128,activation='relu'))
    model.add(Dense(3, activation='softmax'))

    model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 8, 8, 1024)        2098176   
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 1024)        4096      
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 8, 1024)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 512)         2097664   
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 512)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 512)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 256)         524544    
__________

In [13]:
# Combine pre-trained model and customized final layers
if transfer_learning:
    final_model = Sequential(name='Pneumonia Classifier')
    final_model.add(base_model)
    final_model.add(model)
else:
    final_model = model
    
final_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 8, 8, 512)         20024384  
_________________________________________________________________
sequential_1 (Sequential)    (None, 3)                 4988675   
Total params: 25,013,059
Trainable params: 4,986,115
Non-trainable params: 20,026,944
_________________________________________________________________


In [14]:
# Compile the model
if optimizer_type == 'SGD':
    optimizer=optimizers.SGD(lr=learning_rate, momentum=0.9)
elif optimizer_type == 'Adam':
    optimizer = optimizers.Adam(lr=learning_rate)
elif optimizer_type == 'Adadelta':
    optimizer = optimizers.Adadelta() 
else:
    optimizer = optimizers.RMSprop()
    
final_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

The next step is Augmentation and it will take place only if it was set to True in in the parameters.

In [15]:
if Augmentation_Indicator:
    # create and configure augmented image generator
    datagen_train = ImageDataGenerator(
            rotation_range=25,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest')

    # create and configure augmented image generator
    datagen_valid = ImageDataGenerator(
            rotation_range=25,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest')
    # fit augmented image generator on data
    datagen_train.fit(train_tensors)
    datagen_valid.fit(valid_tensors)

In [16]:
# Make the directory to save the models
os.mkdir('/kaggle/working/saved-models')

In [24]:
# Create a pattern to do some analysis later to understand how each parameter affects the model's performance.
if Augmentation_Indicator:
    model_weights_name = 'weights.best.{}_wAug_{}_{}_{}_{}_{}_{}.hd5'.format(learning_name, model_type, dropout, optimizer_type,learning_rate, epochs, batch_size)
else:
    model_weights_name = 'weights.best.{}_woAug_{}_{}_{}_{}_{}_{}.hd5'.format(learning_name, model_type, dropout, optimizer_type,learning_rate, epochs, batch_size)
print(model_weights_name)

weights.best.VGG19TransferLearning_woAug_FC1024_0.4_Adam_0.0001_15_16.hd5


In [18]:
# Fit the model and save the model with best weights
from keras.callbacks import ModelCheckpoint  

checkpointer = ModelCheckpoint(filepath='/kaggle/working/saved-models/{}'.format(model_weights_name), 
                               verbose=1, save_best_only=True)

if Augmentation_Indicator:
    final_model.fit_generator(datagen_train.flow(train_tensors, train_targets, batch_size=batch_size),
                                                steps_per_epoch=train_tensors.shape[0] // batch_size,
                                                epochs=epochs, verbose=1, callbacks=[checkpointer],
                                                validation_data=datagen_valid.flow(valid_tensors, valid_targets, batch_size=batch_size),
                                                validation_steps=valid_tensors.shape[0] // batch_size)
else:
    final_model.fit(train_tensors, train_targets, 
              validation_data=(valid_tensors, valid_targets),
              epochs=epochs, batch_size=batch_size, callbacks=[checkpointer], verbose=1)




Train on 7305 samples, validate on 1827 samples
Epoch 1/15
7305/7305 [==============================] - 148s 20ms/step - loss: 0.8115 - categorical_accuracy: 0.6118 - val_loss: 0.7364 - val_categorical_accuracy: 0.6694

Epoch 00001: val_loss improved from inf to 0.73641, saving model to /kaggle/working/saved-models/weights.best.VGG16TransferLearning_woAug_FC1024_0.4_Adam_0.0001_15_16.hd5
Epoch 2/15
7305/7305 [==============================] - 144s 20ms/step - loss: 0.7213 - categorical_accuracy: 0.6679 - val_loss: 0.7034 - val_categorical_accuracy: 0.6847

Epoch 00002: val_loss improved from 0.73641 to 0.70342, saving model to /kaggle/working/saved-models/weights.best.VGG16TransferLearning_woAug_FC1024_0.4_Adam_0.0001_15_16.hd5
Epoch 3/15
7305/7305 [==============================] - 144s 20ms/step - loss: 0.6790 - categorical_accuracy: 0.6964 - val_loss: 0.7020 - val_categorical_accuracy: 0.6946

Epoch 00003: val_loss improved from 0.70342 to 0.70201, saving model to /kaggle/working/sa

In [19]:
# Load the model with best weights
final_model.load_weights('/kaggle/working/saved-models/{}'.format(model_weights_name))

In [20]:
# Get index of predicted value for each image in test set
predictions = [np.argmax(final_model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# Report test accuracy
test_accuracy = 100*np.sum(np.array(predictions)==np.argmax(test_targets, axis=1))/len(predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 70.7389%


In [21]:
# Print the Classification report
from sklearn.metrics import classification_report
target_names = ['Lung Opacity', 'No Lung Opacity / Not Normal', 'Normal']
print(classification_report(np.array(predictions), np.argmax(test_targets, axis=1), target_names = target_names))

                              precision    recall  f1-score   support

                Lung Opacity       0.70      0.71      0.70       312
No Lung Opacity / Not Normal       0.60      0.67      0.63       360
                      Normal       0.87      0.75      0.80       343

                 avg / total       0.72      0.71      0.71      1015



In [25]:
# Print all the parameters
print(learning_name, total_images,input_shape, model_type, dropout, optimizer_type,learning_rate, epochs, batch_size, test_accuracy, rescale)

VGG19TransferLearning 10147 (256, 256, 3) FC1024 0.4 Adam 0.0001 15 16 70.73891625615764 100
